# 많은 수의 도구를 처리하는 방법

<div class="admonition tip">
    <p class="admonition-title">필수 조건</p>
    <p>
        이 가이드는 다음에 대한 익숙함을 가정합니다:
        <ul>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#tools">
                    도구
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#chat-models/">
                    채팅 모델
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#embedding-models">
                    임베딩 모델
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#vector-stores">
                    벡터스토어
                </a>
            </li>   
            <li>
                <a href="https://python.langchain.com/docs/concepts/#documents">
                    문서
                </a>
            </li>
        </ul>
    </p>
</div> 


호출할 수 있는 도구의 하위 집합은 일반적으로 모델의 재량에 달려 있습니다(많은 제공업체는 사용자가 [도구 선택을 지정하거나 제한](https://python.langchain.com/docs/how_to/tool_choice/)할 수 있도록 합니다). 사용 가능한 도구의 수가 증가함에 따라 토큰 소비를 줄이고 LLM 추론의 오류 소스를 관리하는 데 도움이 되도록 LLM의 선택 범위를 제한할 수 있습니다.

여기서는 모델에 사용할 수 있는 도구를 동적으로 조정하는 방법을 보여줍니다. 결론부터 말하면: [RAG](https://python.langchain.com/docs/concepts/#retrieval) 및 유사한 방법과 같이 사용 가능한 도구를 검색하여 모델 호출 앞에 접두사를 붙입니다. 도구 설명을 검색하는 하나의 구현을 보여주지만 도구 선택의 세부 사항은 필요에 따라 사용자 정의할 수 있습니다.

## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정하겠습니다

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai numpy

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>

## 도구 정의

[S&P 500 지수](https://en.wikipedia.org/wiki/S%26P_500)에 상장된 각 공개 거래 회사에 대해 하나의 도구가 있는 간단한 예제를 고려해 봅시다. 각 도구는 매개변수로 제공된 연도를 기반으로 회사별 정보를 가져옵니다.

먼저 고유 식별자를 각 도구의 스키마와 연결하는 레지스트리를 구성합니다. 도구 호출을 지원하는 채팅 모델에 직접 바인딩할 수 있는 JSON 스키마를 사용하여 도구를 표현합니다.

In [ ]:
import re
import uuid

from langchain_core.tools import StructuredTool


def create_tool(company: str) -> dict:
    """플레이스홀더 도구의 스키마를 생성합니다."""
    # 도구 이름에서 영숫자가 아닌 문자를 제거하고 공백을 밑줄로 대체
    formatted_company = re.sub(r"[^\w\s]", "", company).replace(" ", "_")

    def company_tool(year: int) -> str:
        # 회사와 연도에 대한 정적 수익 정보를 반환하는 플레이스홀더 함수
        return f"{company} had revenues of $100 in {year}."

    return StructuredTool.from_function(
        company_tool,
        name=formatted_company,
        description=f"Information about {company}",
    )


# 시연을 위한 S&P 500 회사의 축약 목록
s_and_p_500_companies = [
    "3M",
    "A.O. Smith",
    "Abbott",
    "Accenture",
    "Advanced Micro Devices",
    "Yum! Brands",
    "Zebra Technologies",
    "Zimmer Biomet",
    "Zoetis",
]

# 각 회사에 대한 도구를 생성하고 고유 UUID를 키로 사용하여 레지스트리에 저장
tool_registry = {
    str(uuid.uuid4()): create_tool(company) for company in s_and_p_500_companies
}

## 그래프 정의

### 도구 선택

상태의 정보(예: 최근 사용자 메시지)를 기반으로 사용 가능한 도구의 하위 집합을 검색하는 노드를 구성합니다. 일반적으로 이 단계에는 [검색 솔루션](https://python.langchain.com/docs/concepts/#retrieval)의 전체 범위를 사용할 수 있습니다. 간단한 솔루션으로 벡터 스토어에 도구 설명의 임베딩을 인덱싱하고 의미론적 검색을 통해 사용자 쿼리를 도구와 연결합니다.

In [4]:
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

tool_documents = [
    Document(
        page_content=tool.description,
        id=id,
        metadata={"tool_name": tool.name},
    )
    for id, tool in tool_registry.items()
]

vector_store = InMemoryVectorStore(embedding=OpenAIEmbeddings())
document_ids = vector_store.add_documents(tool_documents)

### 에이전트와 통합

일반적인 ReAct 에이전트 그래프([빠른 시작](https://langchain-ai.github.io/langgraph/tutorials/introduction/#part-2-enhancing-the-chatbot-with-tools)에서 사용된 것과 같은)를 사용하되 약간의 수정을 가합니다:

- 상태에 `selected_tools` 키를 추가하여 선택된 도구의 하위 집합을 저장합니다;
- 그래프의 진입점을 `select_tools` 노드로 설정하여 상태의 이 요소를 채웁니다;
- `agent` 노드 내에서 선택된 도구의 하위 집합을 채팅 모델에 바인딩합니다.

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


# TypedDict를 사용하여 상태 구조를 정의합니다.
# 메시지 목록(add_messages에 의해 처리됨)과
# 선택된 도구 ID 목록을 포함합니다.
class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]


builder = StateGraph(State)

# 도구 레지스트리에서 사용 가능한 모든 도구를 검색합니다.
tools = list(tool_registry.values())
llm = ChatOpenAI()


# agent 함수는 선택된 도구를 LLM에 바인딩하여
# 현재 상태를 처리합니다.
def agent(state: State):
    # 상태의 selected_tools 목록을 기반으로
    # 도구 ID를 실제 도구에 매핑합니다.
    selected_tools = [tool_registry[id] for id in state["selected_tools"]]
    # 현재 상호 작용을 위해 선택된 도구를 LLM에 바인딩합니다.
    llm_with_tools = llm.bind_tools(selected_tools)
    # 현재 메시지로 LLM을 호출하고 업데이트된 메시지 목록을 반환합니다.
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# select_tools 함수는 사용자의 마지막 메시지 내용을 기반으로 도구를 선택합니다.
def select_tools(state: State):
    last_user_message = state["messages"][-1]
    query = last_user_message.content
    tool_documents = vector_store.similarity_search(query)
    return {"selected_tools": [document.id for document in tool_documents]}


builder.add_node("agent", agent)
builder.add_node("select_tools", select_tools)

tool_node = ToolNode(tools=tools)
builder.add_node("tools", tool_node)

builder.add_conditional_edges("agent", tools_condition, path_map=["tools", "__end__"])
builder.add_edge("tools", "agent")
builder.add_edge("select_tools", "agent")
builder.add_edge(START, "select_tools")
graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # 이것은 몇 가지 추가 종속성이 필요하며 선택 사항입니다
    pass

In [14]:
user_input = "Can you give me some information about AMD in 2022?"

result = graph.invoke({"messages": [("user", user_input)]})

In [15]:
print(result["selected_tools"])

['ab9c0d59-3d16-448d-910c-73cf10a26020', 'f5eff8f6-7fb9-47b6-b54f-19872a52db84', '2962e168-9ef4-48dc-8b7c-9227e7956d39', '24a9fb82-19fe-4a88-944e-47bc4032e94a']


In [16]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Can you give me some information about AMD in 2022?
================================== Ai Message ==================================
Tool Calls:
  Advanced_Micro_Devices (call_CRxQ0oT7NY7lqf35DaRNTJ35)
 Call ID: call_CRxQ0oT7NY7lqf35DaRNTJ35
  Args:
    year: 2022
================================= Tool Message =================================
Name: Advanced_Micro_Devices

Advanced Micro Devices had revenues of $100 in 2022.
================================== Ai Message ==================================

In 2022, Advanced Micro Devices (AMD) had revenues of $100.


## 도구 선택 반복

잘못된 도구 선택으로 인한 오류를 관리하기 위해 `select_tools` 노드를 다시 방문할 수 있습니다. 이를 구현하는 한 가지 옵션은 `select_tools`를 수정하여 상태의 모든 메시지를 사용하여 벡터 스토어 쿼리를 생성하고(예: 채팅 모델 사용) `tools`에서 `select_tools`로 라우팅하는 엣지를 추가하는 것입니다.

아래에서 이 변경 사항을 구현합니다. 시연 목적으로 `select_tools` 노드에 `hack_remove_tool_condition`을 추가하여 초기 도구 선택의 오류를 시뮬레이션합니다. 이는 노드의 첫 번째 반복에서 올바른 도구를 제거합니다. 두 번째 반복에서는 에이전트가 올바른 도구에 액세스할 수 있으므로 실행이 완료됩니다.

<div class="admonition note">
    <p class="admonition-title">LangChain과 함께 Pydantic 사용하기</p>
    <p>
        이 노트북은 Pydantic v2 <code>BaseModel</code>을 사용하며, <code>langchain-core >= 0.3</code>이 필요합니다. <code>langchain-core < 0.3</code>을 사용하면 Pydantic v1과 v2 <code>BaseModel</code>이 혼합되어 오류가 발생합니다.
    </p>
</div>

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.pregel.retry import RetryPolicy

from pydantic import BaseModel, Field


class QueryForTools(BaseModel):
    """추가 도구에 대한 쿼리를 생성합니다."""

    query: str = Field(..., description="추가 도구에 대한 쿼리.")


def select_tools(state: State):
    """대화 상태의 마지막 메시지를 기반으로 도구를 선택합니다.

    마지막 메시지가 사람으로부터 온 것이면 메시지의 내용을
    직접 쿼리로 사용합니다. 그렇지 않으면 시스템 메시지를 사용하여
    쿼리를 구성하고 LLM을 호출하여 도구 제안을 생성합니다.
    """
    last_message = state["messages"][-1]
    hack_remove_tool_condition = False  # 첫 번째 도구 선택에서 오류를 시뮬레이션

    if isinstance(last_message, HumanMessage):
        query = last_message.content
        hack_remove_tool_condition = True  # 잘못된 도구 선택을 시뮬레이션
    else:
        assert isinstance(last_message, ToolMessage)
        system = SystemMessage(
            "Given this conversation, generate a query for additional tools. "
            "The query should be a short string containing what type of information "
            "is needed. If no further information is needed, "
            "set more_information_needed False and populate a blank string for the query."
        )
        input_messages = [system] + state["messages"]
        response = llm.bind_tools([QueryForTools], tool_choice=True).invoke(
            input_messages
        )
        query = response.tool_calls[0]["args"]["query"]

    # 생성된 쿼리를 사용하여 도구 벡터 스토어를 검색
    tool_documents = vector_store.similarity_search(query)
    if hack_remove_tool_condition:
        # 선택에서 올바른 도구를 제거하여 오류를 시뮬레이션
        selected_tools = [
            document.id
            for document in tool_documents
            if document.metadata["tool_name"] != "Advanced_Micro_Devices"
        ]
    else:
        selected_tools = [document.id for document in tool_documents]
    return {"selected_tools": selected_tools}


graph_builder = StateGraph(State)
graph_builder.add_node("agent", agent)
graph_builder.add_node(
    "select_tools", select_tools, retry_policy=RetryPolicy(max_attempts=3)
)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "agent",
    tools_condition,
)
graph_builder.add_edge("tools", "select_tools")
graph_builder.add_edge("select_tools", "agent")
graph_builder.add_edge(START, "select_tools")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # 이것은 몇 가지 추가 종속성이 필요하며 선택 사항입니다
    pass

In [48]:
user_input = "Can you give me some information about AMD in 2022?"

result = graph.invoke({"messages": [("user", user_input)]})

In [49]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Can you give me some information about AMD in 2022?
================================== Ai Message ==================================
Tool Calls:
  Accenture (call_qGmwFnENwwzHOYJXiCAaY5Mx)
 Call ID: call_qGmwFnENwwzHOYJXiCAaY5Mx
  Args:
    year: 2022
================================= Tool Message =================================
Name: Accenture

Accenture had revenues of $100 in 2022.
================================== Ai Message ==================================
Tool Calls:
  Advanced_Micro_Devices (call_u9e5UIJtiieXVYi7Y9GgyDpn)
 Call ID: call_u9e5UIJtiieXVYi7Y9GgyDpn
  Args:
    year: 2022
================================= Tool Message =================================
Name: Advanced_Micro_Devices

Advanced Micro Devices had revenues of $100 in 2022.
================================== Ai Message ==================================

In 2022, AMD had revenues of $100.


## 다음 단계

이 가이드는 도구를 동적으로 선택하기 위한 최소한의 구현을 제공합니다. 가능한 개선 사항 및 최적화가 다양합니다:

- **도구 선택 반복**: 여기서는 `select_tools` 노드를 수정하여 도구 선택을 반복했습니다. 또 다른 옵션은 에이전트에 `reselect_tools` 도구를 제공하여 재량에 따라 도구를 다시 선택할 수 있도록 하는 것입니다.
- **도구 선택 최적화**: 일반적으로 도구 선택을 위해 [검색 솔루션](https://python.langchain.com/docs/concepts/#retrieval)의 전체 범위를 사용할 수 있습니다. 추가 옵션은 다음과 같습니다:
  - 도구를 그룹화하고 그룹에 대해 검색;
  - 채팅 모델을 사용하여 도구 또는 도구 그룹을 선택.